In [ ]:
!pip install openai langchain

In [ ]:
import boto3
import os
import json
import pandas as pd
from openai import AzureOpenAI

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key="My_api_key",
    api_version="2023-03-15-preview",
    azure_endpoint="https://tuteureopenai.openai.azure.com"
)

# Get interpretation from LLM
def get_interpretation(prediction_value):
    if prediction_value <= 0:
        situation = "optimal soil moisture"
    elif 0 < prediction_value <= 0.2:
        situation = "slightly dry soil"
    elif 0.2 < prediction_value < 0.8:
        situation = "moderately dry soil"
    else:
        situation = "very dry soil"
    
    try:
        response = client.chat.completions.create(
            model="gpt-35-turbo",
            messages=[
                {"role": "system", "content": "You are an irrigation advisor. Give very brief recommendations in 5-7 words."},
                {"role": "user", "content": f"Given {situation} with a moisture prediction value of {prediction_value}, provide a very brief irrigation recommendation."}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error getting LLM interpretation: {e}")
        return get_fallback_interpretation(prediction_value)


# Send batch data to S3
def send_to_s3_csv(batch_data):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    s3_key = f"sensor_data_2/sensor_data_{timestamp}.csv"

    df = pd.DataFrame(batch_data)
    csv_data = df.to_csv(index=False)

    s3_client = boto3.client('s3', region_name='us-east-1')
    bucket_name = 'projectkafkabucket'

    try:
        s3_client.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=csv_data.encode('utf-8'),
            ContentType='text/csv'
        )
        print(f"Successfully uploaded to S3 as CSV: {s3_key}")
    except Exception as e:
        print(f"Error uploading CSV to S3: {e}")
